In [24]:
import psycopg2
import pandas as pd

# PostgreSQL connection details
USER = "postgres"
PASSWORD = "Apurba@2024"
DATABASE_CUSTOMER = "customerdb"  
DATABASE_SALES = "salesdb"  
HOST = "localhost"
PORT = "5432"

# Function to execute a query and return results as a Pandas DataFrame
def execute_query(connection, query):
    try:
        # Create a cursor
        cur = connection.cursor()
        # Execute the query
        cur.execute(query)
        # Fetch results
        columns = [desc[0] for desc in cur.description]  # Get column names
        rows = cur.fetchall()  # Get data rows
        # Convert to DataFrame
        df = pd.DataFrame(rows, columns=columns)
        return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

# Query to execute
query_purchases = "SELECT * FROM Purchases;"
query_sales = "SELECT * FROM SalesData;"

# Execute queries and fetch data
try:
    # Connect to CustomerDB
    conn_customer = psycopg2.connect(
        host=HOST,
        dbname=DATABASE_CUSTOMER,
        user=USER,
        password=PASSWORD,
        port=PORT
    )
    purchases_data = execute_query(conn_customer, query_purchases)
    print("Data from 'Purchases' table:")
    print(purchases_data)
    conn_customer.close()

    # Connect to SalesDB
    conn_sales = psycopg2.connect(
        host=HOST,
        dbname=DATABASE_SALES,
        user=USER,
        password=PASSWORD,
        port=PORT
    )
    sales_data = execute_query(conn_sales, query_sales)
    print("\nData from 'SalesData' table:")
    print(sales_data)
    conn_sales.close()

except Exception as e:
    print(f"Error: {e}")


Data from 'Purchases' table:
     purchasedate  customerid purchaseamount   paymentmethod
0      2020-01-22        8502         346.50     Credit Card
1      2020-01-22        5831         364.27      Debit Card
2      2020-01-22        4386         396.48     Credit Card
3      2020-01-22        2554          20.92            Cash
4      2020-01-22        6276         380.20  Digital Wallet
...           ...         ...            ...             ...
2995   2020-05-15        2314         403.74  Digital Wallet
2996   2020-05-15        4430          32.52  Digital Wallet
2997   2020-05-15        3166          83.90     Credit Card
2998   2020-05-15        4913         406.23            Cash
2999   2020-05-15        5289         461.67  Digital Wallet

[3000 rows x 4 columns]

Data from 'SalesData' table:
            date     product  quantitysold totalrevenue
0     2020-01-22  Smartwatch            41       873.14
1     2020-01-22      Tablet            15       196.58
2     2020-01-22

In [25]:
import xmltodict
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['EmployeeDB']
collection = db['Attendance']

# Read and Parse XML File
with open("Employee_Attendance.xml", "r") as file:
    data = xmltodict.parse(file.read())

# Extract and Convert Data
attendance_records = data['EmployeeAttendance']['Attendance']
records = [dict(record) for record in attendance_records]

# Insert Data into MongoDB
collection.insert_many(records)

print(f"Inserted {len(records)} records into MongoDB.")




# 1. Query: Find All Attendance Records for a Specific Employee
result = collection.find({"EmployeeID": "E0084"})
for record in result:
    print(record)

# 2. Query: Count Total Attendance Records
total_records = collection.count_documents({})
print(f"Total attendance records: {total_records}")

# 3. Query: Employees with Late Check-In (After 9:00 AM)
late_checkins = collection.find({"CheckInTime": {"$gt": "09:00:00"}})
for record in late_checkins:
    print(record)

# 4. Query: Aggregate Total Hours Worked by Each Employee
from datetime import datetime
from bson.son import SON

pipeline = [
    {
        "$project": {
            "EmployeeID": 1,
            "HoursWorked": {
                "$divide": [
                    {
                        "$subtract": [
                            {"$dateFromString": {"dateString": {"$concat": ["2020-01-01T", "$CheckOutTime"]}}},
                            {"$dateFromString": {"dateString": {"$concat": ["2020-01-01T", "$CheckInTime"]}}}
                        ]
                    },
                    3600000
                ]
            }
        }
    },
    {
        "$group": {
            "_id": "$EmployeeID",
            "TotalHours": {"$sum": "$HoursWorked"}
        }
    },
    {"$sort": SON([("TotalHours", -1)])}
]

hours_worked = list(collection.aggregate(pipeline))
for record in hours_worked:
    print(record)

# 5. Query: Find Employees Who Worked Overtime (More Than 8 Hours)
overtime_query = [
    {
        "$project": {
            "EmployeeID": 1,
            "CheckInTime": 1,
            "CheckOutTime": 1,
            "HoursWorked": {
                "$divide": [
                    {
                        "$subtract": [
                            {"$dateFromString": {"dateString": {"$concat": ["2020-01-01T", "$CheckOutTime"]}}},
                            {"$dateFromString": {"dateString": {"$concat": ["2020-01-01T", "$CheckInTime"]}}}
                        ]
                    },
                    3600000
                ]
            }
        }
    },
    {"$match": {"HoursWorked": {"$gt": 8}}},
]

overtime_employees = list(collection.aggregate(overtime_query))
for record in overtime_employees:
    print(record)

# 6. Query: Attendance Records for a Specific Month
april_records = collection.find({"Date": {"$regex": "^2020-04"}})
for record in april_records:
    print(record)

# 7. Query: Count Records Grouped by Date
pipeline = [
    {
        "$group": {
            "_id": "$Date",
            "TotalEmployees": {"$sum": 1}
        }
    },
    {"$sort": {"_id": 1}}
]

daily_attendance = list(collection.aggregate(pipeline))
for record in daily_attendance:
    print(record)





Inserted 3000 records into MongoDB.
{'_id': ObjectId('67532490b9c91cfc2c43e5b7'), 'Date': '2020-04-24', 'EmployeeID': 'E0084', 'CheckInTime': '09:23:00', 'CheckOutTime': '16:03:00'}
{'_id': ObjectId('67532490b9c91cfc2c43e5bc'), 'Date': '2020-01-24', 'EmployeeID': 'E0084', 'CheckInTime': '08:22:00', 'CheckOutTime': '16:13:00'}
{'_id': ObjectId('67532490b9c91cfc2c43e638'), 'Date': '2020-05-08', 'EmployeeID': 'E0084', 'CheckInTime': '08:43:00', 'CheckOutTime': '16:49:00'}
{'_id': ObjectId('67532490b9c91cfc2c43e65e'), 'Date': '2020-03-18', 'EmployeeID': 'E0084', 'CheckInTime': '08:02:00', 'CheckOutTime': '16:44:00'}
{'_id': ObjectId('67532490b9c91cfc2c43e69b'), 'Date': '2020-04-23', 'EmployeeID': 'E0084', 'CheckInTime': '09:12:00', 'CheckOutTime': '16:26:00'}
{'_id': ObjectId('67532490b9c91cfc2c43e6b9'), 'Date': '2020-03-13', 'EmployeeID': 'E0084', 'CheckInTime': '08:58:00', 'CheckOutTime': '17:21:00'}
{'_id': ObjectId('67532490b9c91cfc2c43e6c4'), 'Date': '2020-03-26', 'EmployeeID': 'E0084

In [26]:
import xml.etree.ElementTree as ET
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['StoreInventoryDB']
collection = db['InventoryUpdates']

# Parse XML File
tree = ET.parse('Store_Inventory_Updates.xml')
root = tree.getroot()

# Extract and Prepare Data for MongoDB
updates = []
for update in root.findall('Update'):
    record = {child.tag: child.text for child in update}
    updates.append(record)

# Insert Data into MongoDB
collection.insert_many(updates)

print(f"Inserted {len(updates)} records into MongoDB.")

# 1. Query: Get All Updates for a Specific Product
laptop_updates = collection.find({"Product": "Laptop"})
for update in laptop_updates:
    print(update)

# 2. Query: Total Stock Added and Removed for Each Product
pipeline = [
    {
        "$group": {
            "_id": "$Product",
            "TotalStockIn": {"$sum": {"$toInt": "$StockIn"}},
            "TotalStockOut": {"$sum": {"$toInt": "$StockOut"}}
        }
    }
]

stock_summary = list(collection.aggregate(pipeline))
for product in stock_summary:
    print(product)

# 3. Query: Products with More Stock Removed Than Added
pipeline = [
    {
        "$group": {
            "_id": "$Product",
            "TotalStockIn": {"$sum": {"$toInt": "$StockIn"}},
            "TotalStockOut": {"$sum": {"$toInt": "$StockOut"}}
        }
    },
    {"$match": {"$expr": {"$gt": ["$TotalStockOut", "$TotalStockIn"]}}}
]

stock_deficit = list(collection.aggregate(pipeline))
for product in stock_deficit:
    print(product)

# 4. Query: Stock Changes Over Time for a Product
tablet_updates = collection.find({"Product": "Tablet"}, {"Date": 1, "StockIn": 1, "StockOut": 1})
for update in tablet_updates:
    print(update)

# 5. Query: Count of Updates Per Product
pipeline = [
    {
        "$group": {
            "_id": "$Product",
            "UpdateCount": {"$sum": 1}
        }
    }
]

product_update_counts = list(collection.aggregate(pipeline))
for count in product_update_counts:
    print(count)


Inserted 3000 records into MongoDB.
{'_id': ObjectId('6753d6b274202861dffa8d4e'), 'Date': '2020-04-18', 'Product': 'Laptop', 'StockIn': '36', 'StockOut': '15'}
{'_id': ObjectId('6753d6b274202861dffa8d4f'), 'Date': '2020-04-11', 'Product': 'Laptop', 'StockIn': '15', 'StockOut': '41'}
{'_id': ObjectId('6753d6b274202861dffa8d5b'), 'Date': '2020-01-22', 'Product': 'Laptop', 'StockIn': '41', 'StockOut': '39'}
{'_id': ObjectId('6753d6b274202861dffa8d62'), 'Date': '2020-02-12', 'Product': 'Laptop', 'StockIn': '52', 'StockOut': '16'}
{'_id': ObjectId('6753d6b274202861dffa8d69'), 'Date': '2020-03-31', 'Product': 'Laptop', 'StockIn': '42', 'StockOut': '29'}
{'_id': ObjectId('6753d6b274202861dffa8d6e'), 'Date': '2020-02-28', 'Product': 'Laptop', 'StockIn': '92', 'StockOut': '49'}
{'_id': ObjectId('6753d6b274202861dffa8d76'), 'Date': '2020-03-17', 'Product': 'Laptop', 'StockIn': '35', 'StockOut': '22'}
{'_id': ObjectId('6753d6b274202861dffa8d78'), 'Date': '2020-03-05', 'Product': 'Laptop', 'StockI